In [13]:
#compute the eigenvectors of Schur's matrix according to Morton's '78 paper, "On the Eigenvectors of Schur's Matrix"
#https://www.sciencedirect.com/science/article/pii/0022314X80900839

In [14]:
#compute the eigenvectors of Schur's matrix (symbolically) over the complex numbers (or a cyclotomic field containing q^th roots of unity)
#note: A is not unitary 
q=5
alpha = exp(2*pi*I/q)
#K.<alpha> = CyclotomicField(3)
A = matrix([[(alpha**(i*j)).expand() for j in range(1,q+1)] for i in range(1,q+1)])
A_eigs = A.eigenvectors_right(); A_eigs

[(sqrt(5),
  [(1, 0, 0, 1, 1/2*sqrt(5) + 1/2), (0, 1, 1, 0, 1/2*sqrt(5) + 1/2)],
  2),
 (-I*sqrt(5),
  [(1, -1/2*sqrt(5) - 1/2*sqrt(2*sqrt(5) + 10) - 1/2, 1/2*sqrt(5) + 1/2*sqrt(2*sqrt(5) + 10) + 1/2, -1, 0)],
  1),
 (I*sqrt(5),
  [(1, -1/2*sqrt(5) + 1/2*sqrt(2*sqrt(5) + 10) - 1/2, 1/2*sqrt(5) - 1/2*sqrt(2*sqrt(5) + 10) + 1/2, -1, 0)],
  1),
 (-sqrt(5), [(1, 1, 1, 1, -sqrt(5) + 1)], 1)]

In [15]:
#compute the eigenvectors of the DFT matrix (symbolically) over the complex numbers (or a cyclotomic field containing q^th roots of unity)
#NOTE: The DFT matrix is a cyclic permutation of rows and cols applied to Schur's matrix
DFT = matrix([[(alpha**(i*j)).expand() for j in range(q)] for i in range(q)])
DFT.eigenvectors_right()

[(sqrt(5),
  [(1, 0, 1/2*sqrt(5) - 1/2, 1/2*sqrt(5) - 1/2, 0), (0, 1, -1, -1, 1)],
  2),
 (-I*sqrt(5),
  [(0, 1, -1/2*sqrt(5) - 1/2*sqrt(2*sqrt(5) + 10) - 1/2, 1/2*sqrt(5) + 1/2*sqrt(2*sqrt(5) + 10) + 1/2, -1)],
  1),
 (I*sqrt(5),
  [(0, 1, -1/2*sqrt(5) + 1/2*sqrt(2*sqrt(5) + 10) - 1/2, 1/2*sqrt(5) - 1/2*sqrt(2*sqrt(5) + 10) + 1/2, -1)],
  1),
 (-sqrt(5),
  [(1, -1/4*sqrt(5) - 1/4, -1/4*sqrt(5) - 1/4, -1/4*sqrt(5) - 1/4, -1/4*sqrt(5) - 1/4)],
  1)]

In [16]:
#check that A is similar to the DFT matrix by cyclicly permuting columns and rows
sigma = Permutation([(i % q)+1 for i in range(1,q+1)])
P = sigma.to_matrix()
P.inverse()*DFT*P == A

True

In [17]:
#define the vectors X_d given a divisor d of q/f(chi), where f(chi) is the conductor of the character \chi
#characters are evaluated as primitive characeters, i.e. relative to the conductor
def X(d,chi):
    q = chi.modulus()
    f = chi.conductor()
    assert d.divides(q/f)
    return vector([chi.primitive_character()(n/d) if d.divides(n) else 0 for n in range(1,q+1)])

In [18]:
#compute eigenvectors given a character chi and a divisor d of q/f(chi)
def eigenvectors(d,chi):
    q = chi.modulus()
    f = chi.conductor()
    try:
        assert d.divides(q/f)
    except AssertionError:
        print("Error: d must divide q/f(chi)")
        return

    #define the eigenvalues
    eig_plus = sqrt(chi(-1)*q)
    eig_minus = -sqrt(chi(-1)*q)
    eig_X = sqrt(q/f)*chi.primitive_character().gauss_sum()
    #map the eigenvalue to the symbolic ring and simplify
    eig_X = SR(eig_X).simplify_full()

    #define the eigenvectors E(chi,d,lambda) as in (13) in the paper
    #need to restrict character to modulus conductor, i.e. the primitive character|
    def E(chi,d,eig):
        conj_coeff = eig/(chi.bar().primitive_character().gauss_sum()*sqrt(d))
        conj_coeff = SR(conj_coeff).full_simplify()
        conj_divisor = Integer(q/(f*d))
        return sqrt(d)*X(d,chi) + conj_coeff*X(conj_divisor,chi.bar())

    #check if chi is real and if the ratio of modulus to conductor is the square of a divisor
    reality_condition = chi.bar() == chi
    square_condition = d^2 == q/f

    #if chi is real and square condition is met, just return X_d(chi)
    if reality_condition and square_condition:
        return [(eig_X, X(d,chi))]

    #if one or the other condition is met, return a pair of eigenvalues corresponding to E(chi,d,\pm lambda)
    if reality_condition != square_condition:
        return (eig_plus, E(chi,d,eig_plus)), (eig_minus, E(chi,d,eig_minus))

    #if neither condition are met, return four eigenvalues E(chi,d,\pm lambda) and E(chi_bar,d,\pm lambda)
    if (not reality_condition) and (not square_condition):
        return (eig_plus,E(chi,d,eig_plus)), (eig_minus,E(chi,d,eig_minus)), (eig_plus,E(chi.bar(),d,eig_plus)), (eig_minus,E(chi.bar(),d,eig_minus))

In [19]:
from sage.misc.flatten import flatten
#compute all the eigenvectors the Schur matrix
#return a dict with eigenvalues as keys and list of eigenvectors as values
def all_eigenvectors(q):
    all_eigs = {}
    for chi in DirichletGroup(q):
        f = chi.conductor()
        for d in divisors(q/f):
            for v in eigenvectors(Integer(d),chi):
                if v[0] not in all_eigs:
                    all_eigs[v[0]] = [v[1]]
                else:
                    all_eigs[v[0]].append(v[1])
    return all_eigs

In [20]:
#ISSUE 1: dependent eigenvectors in list
#EXPECTED BEHAVIOR: the number of independent eigenvectors should be exactly q
#for q=3 and q=4, it seems that the second condition is being called twice when it should only be called once
#it's giving two independent eigenvectors each time, but they are dependent on other vectors in the list
#for q=4, we are computing d=1, conj_divisor=4 and d=4, conj_divisor=1 and getting eigenvectors which are dependent
#when we look at the sum over characters and divisors of 1, we get q. that means there would only be 1 eigenvector for the pair (chi,d)
#but there are two in one case, four in the other case
sum_total = 0
for chi in DirichletGroup(q):
    f = chi.conductor()
    for d in divisors(q/f):
        #if we contribute 2 or 4 eigenvectors here, the total number will be wrong
        sum_total += 1
print(sum_total==q)

True


In [21]:
all_eigenvectors(4)

{2: [(1, 1, 1, 3), (0, 1, 0, 1), (1, 1, 1, 3)],
 -2: [(1, 1, 1, -1), (-1, -1, -1, 1)],
 2*I: [(1, 0, -1, 0)]}